# Churn Risk Prediction — Logistic Regression (Baseline Model)

## Objective
Build a simple, interpretable baseline model to predict customer churn risk
using behavioral features such as spending, purchase frequency, recency, and loyalty.




## 1. Load Processed Feature Dataset


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Load processed dataset
df = pd.read_csv("../data/processed/customer_features_ml.csv")

df.head()


,customer_id,age,total_spend,items_purchased,avg_rating,discount_applied,days_since_last_purchase,satisfaction_level,spend_segment,frequency_segment,...,recency_norm,loyalty_score,gender_Male,membership_type_Gold,membership_type_Silver,city_Houston,city_Los Angeles,city_Miami,city_New York,city_San Francisco
0,101,29,1120.20,14,4.6,True,25,Satisfied,High Spend,Medium Frequency,...,0.703704,0.616912,False,True,False,False,False,False,True,False
1,102,34,780.50,11,4.1,False,18,Neutral,Medium Spend,Medium Frequency,...,0.833333,0.469024,True,False,True,False,True,False,False,False
2,103,43,510.75,9,3.4,True,42,Unsatisfied,Low Spend,Low Frequency,...,0.388889,0.195565,False,False,False,False,False,False,False,False
3,104,30,1480.30,19,4.7,False,12,Satisfied,High Spend,High Frequency,...,0.944444,0.926125,True,True,False,False,False,False,False,True
4,105,27,720.40,13,4.0,True,55,Unsatisfied,Medium Spend,Medium Frequency,...,0.148148,0.284654,True,False,True,False,False,True,False,False


## 2. Target Variable

- `churn_risk = 1` → High churn risk  
- `churn_risk = 0` → Low churn risk


In [4]:
df["churn_risk"].value_counts(normalize=True)


churn_risk
0    0.651429
1    0.348571
Name: proportion, dtype: float64

## 3. Feature Selection (Model v1)

We intentionally select a small set of meaningful numeric features to:
- Keep the model interpretable
- Avoid noise and overfitting
- Reflect real business logic

This is a baseline model, not a final one.


In [5]:
features = [
    "total_spend",
    "items_purchased",
    "days_since_last_purchase",
    "avg_rating",
    "loyalty_score",
    "discount_applied",
    "membership_type_Gold",
    "membership_type_Silver"
]

X = df[features]
y = df["churn_risk"]

X.head()


,total_spend,items_purchased,days_since_last_purchase,avg_rating,loyalty_score,discount_applied,membership_type_Gold,membership_type_Silver
0,1120.20,14,25,4.6,0.616912,True,True,False
1,780.50,11,18,4.1,0.469024,False,False,True
2,510.75,9,42,3.4,0.195565,True,False,False
3,1480.30,19,12,4.7,0.926125,False,True,False
4,720.40,13,55,4.0,0.284654,True,False,True


## 4. Train-Test Split

We split the data to evaluate model generalization.


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


## 5. Feature Scaling

Logistic regression benefits from feature scaling, especially when features
have different ranges (e.g. spend vs ratings).


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 6. Train Logistic Regression Model


In [8]:
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

## 7. Model Evaluation


In [9]:
y_pred = log_model.predict(X_test_scaled)
y_prob = log_model.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99        57
           1       1.00      0.97      0.98        31

    accuracy                           0.99        88
   macro avg       0.99      0.98      0.99        88
weighted avg       0.99      0.99      0.99        88



In [10]:
roc_auc_score(y_test, y_prob)


1.0

## 8. Confusion Matrix


In [11]:
confusion_matrix(y_test, y_pred)


array([[57,  0],
       [ 1, 30]])

## 9. Feature Importance Interpretation

Logistic regression coefficients indicate the direction and strength
of each feature’s influence on churn risk.


In [12]:
coefficients = pd.DataFrame({
    "feature": features,
    "coefficient": log_model.coef_[0]
}).sort_values(by="coefficient", ascending=False)

coefficients


,feature,coefficient
2,days_since_last_purchase,2.952939
5,discount_applied,1.989225
7,membership_type_Silver,0.592592
1,items_purchased,-0.069631
3,avg_rating,-0.091645
0,total_spend,-0.263666
6,membership_type_Gold,-0.531423
4,loyalty_score,-1.003278


## Business Insights from Churn Model

- Customer inactivity (days since last purchase) is the strongest predictor of churn risk, confirming the importance of recency-based monitoring.
- Discount-driven customers show significantly higher churn risk, suggesting that promotions alone may attract price-sensitive customers without long-term loyalty.
- Loyalty score and Gold membership status strongly reduce churn risk, highlighting the effectiveness of loyalty programs.
- Higher-spending and more engaged customers are less likely to churn, supporting prioritization of high-value customer retention strategies.

Overall, the model validates key behavioral signals used in customer lifecycle management and provides a strong baseline for retention analytics.


## Decision Tree Model (Comparison)

To validate churn patterns identified by logistic regression, we train a shallow
decision tree model. The goal is interpretability and pattern confirmation,
not maximizing performance.

The tree is intentionally constrained to avoid overfitting.


In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [14]:
tree_model = DecisionTreeClassifier(
    max_depth=3,
    min_samples_leaf=10,
    random_state=42
)

tree_model.fit(X_train, y_train)


,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",3
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",10
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current n

In [15]:
y_pred_tree = tree_model.predict(X_test)
y_prob_tree = tree_model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred_tree))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        57
           1       1.00      1.00      1.00        31

    accuracy                           1.00        88
   macro avg       1.00      1.00      1.00        88
weighted avg       1.00      1.00      1.00        88



In [16]:
roc_auc_score(y_test, y_prob_tree)


1.0

In [17]:
confusion_matrix(y_test, y_pred_tree)


array([[57,  0],
       [ 0, 31]])

## Logistic Regression vs Decision Tree — Comparison

Both models show strong performance due to the recency-based churn definition.
However, their agreement on key churn drivers is more important than the raw scores.

### Observations:
- Both models identify customer inactivity as the dominant churn signal.
- Loyalty-related features consistently reduce churn risk.
- Discount usage contributes to higher churn risk.
- The decision tree captures simple, intuitive rules, while logistic regression
  provides coefficient-based interpretability.

The consistency across models increases confidence in the behavioral insights.
